In [153]:
# community_analysis.ipynb

# 📌 SECTION 1: IMPORTS
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from scipy.stats import chi2_contingency, kruskal
import os
import warnings
import json
warnings.filterwarnings("ignore")

In [154]:
# 📌 SECTION 2: LOAD DATA
base_path = "../data/full"
algo = "reverse_hybrid"
# algo = "w_slpa"
patient_df = pd.read_csv("../../data/thesis/cll_broad_2022_clinical_data_thesis.csv")
mutation_df = pd.read_csv("../../data/thesis/cll_broad_2022_mutations_thesis.csv")
community_df = pd.read_csv(os.path.join(base_path, algo, "community_assignments.csv"))

In [155]:
community_df.head(
)

,nodeId,patientId,communityId
0,740,P-SCLL-0202,0
1,297,P-GCLL-0145,0
2,306,P-GCLL-0156,0
3,406,P-GCLL-0266,0
4,658,P-SCLL-0103,0


In [156]:
# make mapping of the following format
# 
# {
#   "0": {"label": "Community 0", "nodes": 156},
#   "1": {"label": "Community 1", "nodes": 80},
#   ...
#   "9": {"label": "Community 9", "nodes": 148}
# }

community_mapping = {}
for community in community_df["communityId"].unique():
    community_nodes = community_df[community_df["communityId"] == community]["nodeId"].tolist()
    community_mapping[str(community)] = {
        "label": f"Community {community}",
        "nodes": len(community_nodes)
    }

# print("Community Mapping:")
# print(json.dumps(community_mapping, indent=2))


In [159]:
import xml.etree.ElementTree as ET

# Load the SVG file (assumed to be named 'leiden_level1.svg')
svg_file = "../../figures/graphs/level1.svg"
tree = ET.parse(svg_file)
root = tree.getroot()

# Define SVG namespace
ns = {'svg': 'http://www.w3.org/2000/svg'}
ET.register_namespace('', ns['svg'])

# Define ellipse parameters for each community (x, y, rx, ry, rotation)
# These are placeholder values and should be adjusted manually as needed
ellipse_params = {
    "0": {"cx": -390, "cy": 250, "rx": 200, "ry": 200, "rotation": 0},
    "1": {"cx": -100, "cy": 280, "rx": 100, "ry": 120, "rotation": 0},
    "2": {"cx": -100, "cy": 510, "rx": 200, "ry": 120, "rotation": 0},
    "4": {"cx": 360, "cy": 425, "rx": 125, "ry": 75, "rotation": 30},
    "3": {"cx": 200, "cy": 450, "rx": 125, "ry": 50, "rotation": 60},
    "5": {"cx": 240, "cy": 85, "rx": 150, "ry": 275, "rotation": 35},
    "6": {"cx": 450, "cy": 275, "rx": 175, "ry": 100, "rotation": 0},
    "7": {"cx": 540, "cy": -20, "rx": 125, "ry": 225, "rotation": 0},
    "8": {"cx": 80, "cy": -350, "rx": 300, "ry": 280, "rotation": 0},
    "9": {"cx": -390, "cy": -250, "rx": 210, "ry": 230, "rotation": 0}
}

# y = -600 to +600
# x = -600 to +600

# Add ellipses and labels to the SVG
for cid, params in ellipse_params.items():
    label = community_mapping[cid]["label"]
    nodes = community_mapping[cid]["nodes"]
    cx, cy, rx, ry, rot = params["cx"], params["cy"], params["rx"], params["ry"], params["rotation"]

    # Create ellipse element
    ellipse = ET.Element('ellipse', {
        'cx': str(cx),
        'cy': str(cy),
        'rx': str(rx),
        'ry': str(ry),
        'transform': f'rotate({rot},{cx},{cy})',
        'fill': 'none',
        'stroke': 'grey',
        'stroke-width': '8'
    })

    # Create text label
    text = ET.Element('text', {
        'x': str(cx),
        'y': str(cy),
        'font-size': '35',
        'text-anchor': 'middle',
        'fill': 'black',
        # font thickness
        'font-weight': 'bold'
    })
    text.text = f"{cid} (n={nodes})"

    # Append to root
    root.append(ellipse)
    root.append(text)

# Save the annotated SVG
output_svg = "annotated_leiden_level1.svg"
tree.write(output_svg)

print(f"Annotated SVG saved as {output_svg}")

Annotated SVG saved as annotated_leiden_level1.svg
